In [ ]:
from time import sleep 
import pywinusb.hid as hid

In [ ]:
"""select the func and parameters"""

teVendorId  = 0x168C
teLucidDesktopId  = 0x6002  # Use this for Lucid Desktop - 6GHz
teLucidPortableId = 0x6081  # Use this for Lucid Portable - 6GHz + 1 Channel
teLucidBenchtopId = 0x1202  # Use this for Lucid Benchtop - 12GHz + 2 Channels
BUFFER_SIZE = 256

class test():
    def __init__(self):
        self.text = None 

    def readData(self,data):
        strData = ''.join([str(elem) for i, elem in enumerate(data)])
        text = ""
        if len(data) == 0:
            print("Data is empty")
        elif strData: #and not strData.isspace() and strData.isprintable():
            for c in data:
                if c != 0:
                    text+= chr(c)
            print (text)
            self.text = text
        return None

    def send_scpi_cmd(self, device, scpi_cmd):
        if not device:
            print ("No device provided")
            return

        device.open()
        buffer=[0x00]*BUFFER_SIZE # USB packet size
        sendData = bytearray(scpi_cmd, 'utf-8')
        sendData_len = len(scpi_cmd)

        for  i in range(0, sendData_len):
            buffer[i+3] = sendData[i]

        device.send_output_report(buffer)
        sleep(0.1)
        device.set_raw_data_handler(self.readData)
        device.close()
        return self.text

In [ ]:
''' conecte the device '''

lucid_device = hid.HidDeviceFilter(vendor_id=teVendorId, product_id=teLucidBenchtopId).get_devices()[0]

In [ ]:
''' Reset the device and test the connection '''

a = test()
a.send_scpi_cmd(lucid_device, '*CLS')
a.send_scpi_cmd(lucid_device, '*RST')
a.send_scpi_cmd(lucid_device,'*IDN?')

In [ ]:
""" set the parameters of sweep mode"""

start = 1e9
step = 5
stop = 3e9
time = 0.5  # works only in continous mode(without external trigger)
pow = -10

a.send_scpi_cmd(lucid_device, ':INST 1\n') # select the channel
a.send_scpi_cmd(lucid_device, ':INIT:GATE OFF')
# a.send_scpi_cmd(lucid_device, ':INIT:CONT ON')
a.send_scpi_cmd(lucid_device, ':TRIG:SOUR EXTernal')
a.send_scpi_cmd(lucid_device, ':TRIG:EDG POS')
# a.send_scpi_cmd(lucid_device, ':TRIG:ADV STEP')

print('the state of gate :')
a.send_scpi_cmd(lucid_device, ':INIT:GATE?')
print('the state of continous mode :')
a.send_scpi_cmd(lucid_device, ':INIT:CONT?')
print('the state of source :')
a.send_scpi_cmd(lucid_device, ':TRIG:SOUR?')
print('the state of trigger edge :')
a.send_scpi_cmd(lucid_device, ':TRIG:EDG?')
print('the state of trigger adv :')
a.send_scpi_cmd(lucid_device, ':TRIG:ADV?')


a.send_scpi_cmd(lucid_device, 'POWer {}\n'.format(pow))
a.send_scpi_cmd(lucid_device, ':FRSW:STAR {}\n'.format(start))
a.send_scpi_cmd(lucid_device, ':FRSW:STEP {}\n'.format(step))
a.send_scpi_cmd(lucid_device, ':FRSW:STOP {}\n'.format(stop))
a.send_scpi_cmd(lucid_device, ':FRSW:TIME {}\n'.format(time))
a.send_scpi_cmd(lucid_device, ':FRSW:DIR NORMal\n')

# a.send_scpi_cmd(lucid_device, ':FRSW ON')
print('the ON/OFF of FRSW :')
a.send_scpi_cmd(lucid_device, ':FRSW?')

In [ ]:
# set the parameters of single freq output

freq = int(1.0003e9)
pow = -10
a.send_scpi_cmd(lucid_device, 'FREQuency {}\n'.format(freq))
a.send_scpi_cmd(lucid_device, 'POWer {}\n'.format(pow))


In [ ]:
""" test 1: check if there`re any munual influence between parameters of sweep mode and single freq output """

print('the value of power :')
a.send_scpi_cmd(lucid_device, 'POWer?')
print('the value of freq :')
a.send_scpi_cmd(lucid_device, 'FREQuency?')
print('the value of freq-start :')
a.send_scpi_cmd(lucid_device, ':FRSW:STAR?')
print('the value of freq-step :')
a.send_scpi_cmd(lucid_device, ':FRSW:STEP?')
print('the value of freq-stop :')
a.send_scpi_cmd(lucid_device, ':FRSW:STOP?')


In [ ]:
""" test 2: single frequnency output """

a.send_scpi_cmd(lucid_device, ':FRSW OFF')

print('the ON/OFF of FRSW :')
a.send_scpi_cmd(lucid_device, ':FRSW?')

a.send_scpi_cmd(lucid_device, 'OUTP ON')
sleep(8)
a.send_scpi_cmd(lucid_device, 'OUTP OFF')

In [ ]:
""" test 3: sweep mode under continous mode """

a.send_scpi_cmd(lucid_device, ':INIT:CONT ON')
a.send_scpi_cmd(lucid_device, ':TRIG:ADV ONC')
a.send_scpi_cmd(lucid_device, ':FRSW ON')

print('the state of continous mode :')
a.send_scpi_cmd(lucid_device, ':INIT:CONT?')
print('the ON/OFF of FRSW :')
a.send_scpi_cmd(lucid_device, ':FRSW?')
print('the state of source :')
a.send_scpi_cmd(lucid_device, ':TRIG:SOUR?')
print('the state of adv :')
a.send_scpi_cmd(lucid_device, ':TRIG:ADV?')

a.send_scpi_cmd(lucid_device, ':OUTPut ON\n')
sleep(5)
a.send_scpi_cmd(lucid_device, ':OUTPut OFF\n')
a.send_scpi_cmd(lucid_device, ':FRSW OFF')

In [ ]:
""" test 4: sweep mode using trigger """

a.send_scpi_cmd(lucid_device, ':INIT:CONT OFF')
a.send_scpi_cmd(lucid_device, ':INIT:GATE OFF')
a.send_scpi_cmd(lucid_device, ':TRIG:ADV STEP')
a.send_scpi_cmd(lucid_device, ':TRIG:SOUR EXT')
a.send_scpi_cmd(lucid_device, ':FRSW ON')

print('the state of continous mode :')
a.send_scpi_cmd(lucid_device, ':INIT:CONT?')
print('the state of continous mode :')
a.send_scpi_cmd(lucid_device, ':INIT:CONT?')
print('the state of continous mode :')
a.send_scpi_cmd(lucid_device, ':TRIG:ADV?')
print('the state of source :')
a.send_scpi_cmd(lucid_device, ':TRIG:SOUR?')
print('the ON/OFF of FRSW :')
a.send_scpi_cmd(lucid_device, ':FRSW?')

In [ ]:
a.send_scpi_cmd(lucid_device, ':OUTPut ON\n')

In [ ]:
a.send_scpi_cmd(lucid_device, ':OUTPut OFF\n')
# a.send_scpi_cmd(lucid_device, ':FRSW OFF')

In [129]:
""" test 5: list mode without trigger """

# initializition of list parameters
a.send_scpi_cmd(lucid_device, ':LIST:DELete:ALL')
freq_list=[[1,2,3,4],[5e8,10e8,20e8,30e8],[-10,-10,-10,-10],[0,0,0,1],[0,0,0,0], [2,5,2,3]]
for i in range(len(freq_list[0])):
        a.send_scpi_cmd(lucid_device, 'LIST:DEFine {},{},{},{},{},{}'.format(
                freq_list[0][i], 
                freq_list[1][i],
                freq_list[2][i],
                freq_list[3][i], 
                freq_list[4][i], 
                freq_list[5][i]
                ))
        sleep(0.5)

a.send_scpi_cmd(lucid_device, ':INIT:CONT ON')
a.send_scpi_cmd(lucid_device, ':TRIG:ADV ONC')
a.send_scpi_cmd(lucid_device, ':LIST ON')
a.send_scpi_cmd(lucid_device, ':FRSW OFF')

print('the state of CONT :')
a.send_scpi_cmd(lucid_device, ':INIT:CONT?')
print('the state of ADV :')
a.send_scpi_cmd(lucid_device, ':TRIG:ADV?')
print('the ON/OFF of LIST :')
a.send_scpi_cmd(lucid_device, ':LIST?')
print('the ON/OFF of FRSW :')
a.send_scpi_cmd(lucid_device, ':FRSW?')

a.send_scpi_cmd(lucid_device, ':OUTPut ON\n')
# sleep(9)
# a.send_scpi_cmd(lucid_device, ':OUTPut OFF\n')
# a.send_scpi_cmd(lucid_device, ':LIST OFF')









the state of CONT :
ON
the state of ADV :
ONC
the ON/OFF of LIST :
OFF
the ON/OFF of FRSW :
OFF



''

In [130]:
# sleep(9)
a.send_scpi_cmd(lucid_device, ':OUTPut OFF\n')
a.send_scpi_cmd(lucid_device, ':LIST OFF')

''

In [131]:
""" test 6: list mode using external trigger """

a.send_scpi_cmd(lucid_device, ':INIT:CONT OFF')
a.send_scpi_cmd(lucid_device, ':TRIG:ADV STEP')
a.send_scpi_cmd(lucid_device, ':LIST ON')
a.send_scpi_cmd(lucid_device, ':FRSW OFF')

print('the state of CONT :')
a.send_scpi_cmd(lucid_device, ':INIT:CONT?')
print('the state of ADV :')
a.send_scpi_cmd(lucid_device, ':TRIG:ADV?')
print('the ON/OFF of LIST :')
a.send_scpi_cmd(lucid_device, ':LIST?')
print('the ON/OFF of FRSW :')
a.send_scpi_cmd(lucid_device, ':FRSW?')

a.send_scpi_cmd(lucid_device, ':OUTPut ON\n')
sleep(9)
a.send_scpi_cmd(lucid_device, ':OUTPut OFF\n')
a.send_scpi_cmd(lucid_device, ':LIST OFF')






the state of CONT :
OFF
the state of ADV :
STEP
the ON/OFF of LIST :
OFF
the ON/OFF of FRSW :
OFF




''